# Imports & config

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
import torch.nn.functional as F

In [3]:
os.chdir('C:\\Users\\Usuario\\TFG\\digipanca\\')

# __Function__

In [54]:
# class SegmentationMetrics:
#     """
#     Class for computing segmentation metrics for 2D/3D segmentation.
#     """
#     @staticmethod
#     def _prepare_input(y_pred, y_true):
#         """
#         Prepare inputs for calculating segmentation metrics. Convert indices to one-hot format if needed.
#         """
#         # Debug prints for shapes of y_pred and y_true
#         print(f"Initial y_pred shape: {y_pred.shape}")
#         print(f"Initial y_true shape: {y_true.shape}")
        
#         # Ensure that y_true has the correct number of classes
#         n_classes = torch.max(y_true).item() + 1  # Max value in y_true + 1 for number of classes
#         print(f"n_classes: {n_classes}")

#         if y_pred.dim() == 4:  # 4D tensor for (B, C, H, W)
#             spatial_dims = y_pred.shape[2:]  # All dimensions after the batch and class dimension
            
#             # Create one-hot tensors for y_pred
#             y_pred_indices = torch.argmax(y_pred, dim=1)  # Get the class indices for each pixel (B, H, W)
#             y_pred_one_hot = torch.zeros(y_pred.size(0), n_classes, *spatial_dims, device=y_pred.device)
#             y_pred_one_hot.scatter_(1, y_pred_indices.unsqueeze(1), 1)  # (B, 1, H, W)
            
#             # Ensure y_true has the correct number of dimensions for scatter_
#             if y_true.dim() == 3:  # For 2D (B, H, W)
#                 y_true = y_true.unsqueeze(1)  # Add channel dimension to make it (B, 1, H, W)
            
#             print(f"y_true shape after unsqueeze: {y_true.shape}")  # Debug print
            
#             # Create one-hot tensors for y_true (shape: (B, n_classes, H, W))
#             y_true_one_hot = torch.zeros(y_true.size(0), n_classes, *spatial_dims, device=y_true.device)
#             print(f"y_true_one_hot shape before scatter_: {y_true_one_hot.shape}")  # Debug print
            
#             # Ensure y_true_one_hot has shape (B, n_classes, H, W)
#             y_true_one_hot.scatter_(1, y_true.long(), 1)  # Use the class indices from y_true
            
#             print(f"y_true_one_hot shape after scatter_: {y_true_one_hot.shape}")  # Debug print
            
#         else:
#             # Handle logits or one-hot inputs for 2D/3D
#             if y_pred.dim() == y_true.dim() + 1:  # Logits (B, C, ...) vs indices (B, ...)
#                 n_classes = y_pred.size(1)
#                 y_pred_one_hot = torch.nn.functional.softmax(y_pred, dim=1)
#                 spatial_dims = y_true.shape[1:]
#                 if y_true.dim() == 3:  # If y_true is 2D (B, H, W)
#                     y_true = y_true.unsqueeze(1)  # Add channel dimension
#                 y_true_one_hot = torch.zeros(y_true.size(0), n_classes, *spatial_dims,
#                                               device=y_true.device)
#                 y_true_one_hot.scatter_(1, y_true.long(), 1)
#             else:
#                 y_pred_one_hot = y_pred
#                 y_true_one_hot = y_true
#                 n_classes = y_pred.size(1)

#         return y_pred_one_hot, y_true_one_hot, n_classes

    
#     @staticmethod
#     def dice_coefficient(y_pred, y_true, smooth=1e-6):
#         """
#         Compute Dice coefficient for 2D/3D data.
#         """
#         # Convert to one-hot if inputs are class indices
#         y_pred_one_hot, y_true_one_hot, n_classes = SegmentationMetrics._prepare_input(y_pred, y_true)
        
#         # Calculate Dice for each class
#         dice_scores = []
#         class_dice = {}
#         for i in range(n_classes):
#             pred_class = y_pred_one_hot[:, i, ...]
#             true_class = y_true_one_hot[:, i, ...]
            
#             intersection = torch.sum(pred_class * true_class)
#             union = torch.sum(pred_class) + torch.sum(true_class)
#             dice = (2.0 * intersection + smooth) / (union + smooth)
#             dice_scores.append(dice)
#             class_dice[f"dice_class_{i}"] = dice.item()
        
#         mean_dice = torch.mean(torch.stack(dice_scores))
#         class_dice["dice_mean"] = mean_dice.item()
#         return mean_dice, class_dice
    
#     @staticmethod
#     def iou_score(y_pred, y_true, smooth=1e-6):
#         """
#         Compute IoU for 2D/3D data.
#         """
#         # Same conversion logic as dice_coefficient
#         y_pred_one_hot, y_true_one_hot, n_classes = SegmentationMetrics._prepare_input(y_pred, y_true)
        
#         iou_scores = []
#         class_iou = {}
#         for i in range(n_classes):
#             pred_class = y_pred_one_hot[:, i, ...]
#             true_class = y_true_one_hot[:, i, ...]
            
#             intersection = torch.sum(pred_class * true_class)
#             union = torch.sum(pred_class) + torch.sum(true_class) - intersection
#             iou = (intersection + smooth) / (union + smooth)
#             iou_scores.append(iou)
#             class_iou[f"iou_class_{i}"] = iou.item()
        
#         mean_iou = torch.mean(torch.stack(iou_scores))
#         class_iou["iou_mean"] = mean_iou.item()
#         return mean_iou, class_iou
    
#     @staticmethod
#     def precision_recall(y_pred, y_true, smooth=1e-6):
#         """
#         Compute precision and recall for 2D/3D data.
#         """
#         # Same conversion logic
#         y_pred_one_hot, y_true_one_hot, n_classes = SegmentationMetrics._prepare_input(y_pred, y_true)
        
#         precision_scores = []
#         recall_scores = []
#         class_precision = {}
#         class_recall = {}
#         for i in range(n_classes):
#             pred_class = y_pred_one_hot[:, i, ...]
#             true_class = y_true_one_hot[:, i, ...]
            
#             tp = torch.sum(pred_class * true_class)
#             predicted = torch.sum(pred_class)
#             actual = torch.sum(true_class)
            
#             precision = (tp + smooth) / (predicted + smooth)
#             recall = (tp + smooth) / (actual + smooth)
            
#             precision_scores.append(precision)
#             recall_scores.append(recall)
#             class_precision[f"precision_class_{i}"] = precision.item()
#             class_recall[f"recall_class_{i}"] = recall.item()
        
#         mean_precision = torch.mean(torch.stack(precision_scores))
#         mean_recall = torch.mean(torch.stack(recall_scores))
#         class_precision["precision_mean"] = mean_precision.item()
#         class_recall["recall_mean"] = mean_recall.item()
#         return mean_precision, mean_recall, class_precision, class_recall
    
#     @staticmethod
#     def all_metrics(y_pred, y_true):
#         """
#         Compute all metrics for 2D/3D data.
#         """
#         metrics = {}
        
#         # Convert logits to indices if necessary (no effect on metrics calculation)
#         if y_pred.dim() in (4, 5):  # Logits 2D/3D
#             y_pred_indices = torch.argmax(y_pred, dim=1)
        
#         # Calculate metrics
#         mean_dice, class_dice = SegmentationMetrics.dice_coefficient(y_pred, y_true)
#         metrics.update(class_dice)
        
#         mean_iou, class_iou = SegmentationMetrics.iou_score(y_pred, y_true)
#         metrics.update(class_iou)
        
#         mean_precision, mean_recall, class_precision, class_recall = SegmentationMetrics.precision_recall(y_pred, y_true)
#         metrics.update(class_precision)
#         metrics.update(class_recall)
        
#         metrics['dice'] = mean_dice.item()
#         metrics['iou'] = mean_iou.item()
#         metrics['precision'] = mean_precision.item()
#         metrics['recall'] = mean_recall.item()
        
#         return metrics

In [97]:
import torch

class SegmentationMetrics:
    """
    Clase para calcular métricas de segmentación multiclase en 2D y 3D.
    """
    
    @staticmethod
    def _preprocess_tensors(y_pred, y_true):
        """
        Preprocesa las predicciones y ground truth para cálculo de métricas.
        """
        # Si son logits, convertir a índices de clase
        if y_pred.dim() in [4, 5] and y_pred.size(1) > 1:
            y_pred = torch.argmax(y_pred, dim=1)
        
        # Asegurar que las predicciones y ground truth sean del mismo tipo
        y_pred = y_pred.float()
        y_true = y_true.float()
        
        return y_pred, y_true

    @staticmethod
    def dice_coefficient(y_pred, y_true, smooth=1e-6):
        """
        Calcula el coeficiente Dice para múltiples clases.
        """
        y_pred, y_true = SegmentationMetrics._preprocess_tensors(y_pred, y_true)
        
        # Número de clases
        n_classes = int(y_true.max().item() + 1)
        
        dice_scores = []
        class_dice = {}
        
        for c in range(n_classes):
            # Crear máscaras binarias para la clase actual
            pred_class = (y_pred == c).float()
            true_class = (y_true == c).float()
            
            # Calcular intersección y unión
            intersection = torch.sum(pred_class * true_class)
            union = torch.sum(pred_class) + torch.sum(true_class)
            
            # Calcular Dice
            dice = (2.0 * intersection + smooth) / (union + smooth)
            
            dice_scores.append(dice)
            class_dice[f"dice_class_{c}"] = dice.item()
        
        # Calcular Dice medio
        mean_dice = torch.mean(torch.stack(dice_scores))
        class_dice["dice_mean"] = mean_dice.item()
        
        return mean_dice, class_dice

    @staticmethod
    def iou_score(y_pred, y_true, smooth=1e-6):
        """
        Calcula el Índice de Jaccard (IoU) para múltiples clases.
        """
        y_pred, y_true = SegmentationMetrics._preprocess_tensors(y_pred, y_true)
        
        # Número de clases
        n_classes = int(y_true.max().item() + 1)
        
        iou_scores = []
        class_iou = {}
        
        for c in range(n_classes):
            # Crear máscaras binarias para la clase actual
            pred_class = (y_pred == c).float()
            true_class = (y_true == c).float()
            
            # Calcular intersección y unión
            intersection = torch.sum(pred_class * true_class)
            union = torch.sum(pred_class) + torch.sum(true_class) - intersection
            
            # Calcular IoU
            iou = (intersection + smooth) / (union + smooth)
            
            iou_scores.append(iou)
            class_iou[f"iou_class_{c}"] = iou.item()
        
        # Calcular IoU medio
        mean_iou = torch.mean(torch.stack(iou_scores))
        class_iou["iou_mean"] = mean_iou.item()
        
        return mean_iou, class_iou

    @staticmethod
    def precision_recall(y_pred, y_true, smooth=1e-6):
        """
        Calcula precisión y recall para múltiples clases.
        """
        y_pred, y_true = SegmentationMetrics._preprocess_tensors(y_pred, y_true)
        
        # Número de clases
        n_classes = int(y_true.max().item() + 1)
        
        precision_scores = []
        recall_scores = []
        class_precision = {}
        class_recall = {}
        
        for c in range(n_classes):
            # Crear máscaras binarias para la clase actual
            pred_class = (y_pred == c).float()
            true_class = (y_true == c).float()
            
            # Calcular true positives, predicted positives y actual positives
            true_positives = torch.sum(pred_class * true_class)
            predicted_positives = torch.sum(pred_class)
            actual_positives = torch.sum(true_class)
            
            # Calcular precisión
            precision = (true_positives + smooth) / (predicted_positives + smooth)
            precision_scores.append(precision)
            class_precision[f"precision_class_{c}"] = precision.item()
            
            # Calcular recall
            recall = (true_positives + smooth) / (actual_positives + smooth)
            recall_scores.append(recall)
            class_recall[f"recall_class_{c}"] = recall.item()
        
        # Calcular promedios
        mean_precision = torch.mean(torch.stack(precision_scores))
        mean_recall = torch.mean(torch.stack(recall_scores))
        
        class_precision["precision_mean"] = mean_precision.item()
        class_recall["recall_mean"] = mean_recall.item()
        
        return mean_precision, mean_recall, class_precision, class_recall

    @staticmethod
    def all_metrics(y_pred, y_true):
        """
        Calcula todas las métricas de segmentación.
        """
        metrics = {}
        
        # Calcular métricas
        mean_dice, class_dice = SegmentationMetrics.dice_coefficient(y_pred, y_true)
        mean_iou, class_iou = SegmentationMetrics.iou_score(y_pred, y_true)
        mean_precision, mean_recall, class_precision, class_recall = SegmentationMetrics.precision_recall(y_pred, y_true)
        
        # Agregar métricas de clases
        metrics.update(class_dice)
        metrics.update(class_iou)
        metrics.update(class_precision)
        metrics.update(class_recall)
        
        # Agregar métricas generales
        metrics['dice'] = mean_dice.item()
        metrics['iou'] = mean_iou.item()
        metrics['precision'] = mean_precision.item()
        metrics['recall'] = mean_recall.item()
        
        return metrics

# __Test__

In [5]:
from src.metrics import SegmentationMetrics as sm2d
from src.metrics.segmentation3d import SegmentationMetrics3D as sm3d

In [22]:
def compare_metrics(metrics_a, metrics_b):
    if metrics_a == metrics_b:
        print("Both metrics are identical.")
    else:
        print("Metrics have differences.")

    for key in metrics_a:
        if key in metrics_b:
            va = metrics_a[key]
            vb = metrics_b[key]
            if va != vb:
                print(f"Difference in {key}: a={va}, b={vb}")

    print("Verification completed")

In [70]:
# 2D
y_pred_2d = torch.rand(1, 3, 256, 256)  # 2D logits
y_true_2d = torch.randint(0, 3, (1, 256, 256))  # 2D labels

# 3D
y_pred_3d = torch.rand(1, 3, 64, 256, 256)  # 3D logits
y_true_3d = torch.randint(0, 3, (1, 64, 256, 256))  # 3D labels

# Compute metrics
metrics_2d_a = sm2d.all_metrics(y_pred_2d, y_true_2d)
metrics_2d_b = sm3d.all_metrics(y_pred_2d, y_true_2d)
metrics_3d_a = sm3d.all_metrics(y_pred_3d, y_true_3d)

In [71]:
compare_metrics(metrics_2d_a, metrics_2d_b)

Both metrics are identical.
Verification completed


In [72]:
metrics_2d_c = SegmentationMetrics.all_metrics(y_pred_2d, y_true_2d)
compare_metrics(metrics_2d_a, metrics_2d_c)

Metrics have differences.
Difference in dice_class_0: a=0.33302754163742065, b=0.33385470509529114
Difference in dice_class_1: a=0.3334294557571411, b=0.3377794027328491
Difference in dice_class_2: a=0.33522045612335205, b=0.34432318806648254
Difference in dice_mean: a=0.33389249444007874, b=0.3386524021625519
Difference in iou_class_0: a=0.19977986812591553, b=0.2003755122423172
Difference in iou_class_1: a=0.20006921887397766, b=0.20320972800254822
Difference in iou_class_2: a=0.20136027038097382, b=0.2079652100801468
Difference in iou_mean: a=0.2004031091928482, b=0.2038501501083374
Difference in precision_class_0: a=0.3329533040523529, b=0.3350106179714203
Difference in precision_class_1: a=0.33321183919906616, b=0.33710840344429016
Difference in precision_class_2: a=0.3355134129524231, b=0.3438296318054199
Difference in precision_mean: a=0.333892822265625, b=0.33864954113960266
Difference in recall_class_0: a=0.3331017792224884, b=0.3327067792415619
Difference in recall_class_1: a

In [73]:
metrics_3d_b = SegmentationMetrics.all_metrics(y_pred_3d, y_true_3d)
compare_metrics(metrics_3d_a, metrics_3d_b)

Metrics have differences.
Difference in dice_class_0: a=1.9414256811141968, b=0.3324236571788788
Difference in dice_class_1: a=1.943986415863037, b=0.33311814069747925
Difference in dice_class_2: a=1.9341572523117065, b=0.3339005708694458
Difference in dice_mean: a=1.9398565292358398, b=0.33314743638038635
Difference in iou_class_0: a=33.14467239379883, b=0.19934538006782532
Difference in iou_class_1: a=34.705650329589844, b=0.1998450756072998
Difference in iou_class_2: a=29.375385284423828, b=0.20040856301784515
Difference in iou_mean: a=32.4085693359375, b=0.19986633956432343
Difference in precision_class_0: a=1.0010994672775269, b=0.33249780535697937
Difference in precision_class_1: a=1.0024378299713135, b=0.3331952393054962
Difference in precision_class_2: a=0.997209370136261, b=0.3337491750717163
Difference in precision_mean: a=1.0002487897872925, b=0.33314740657806396
Difference in recall_class_0: a=31.980545043945312, b=0.3323495388031006
Difference in recall_class_1: a=32.00453

In [74]:
print(metrics_3d_b)

{'dice_class_0': 0.3324236571788788, 'dice_class_1': 0.33311814069747925, 'dice_class_2': 0.3339005708694458, 'dice_mean': 0.33314743638038635, 'iou_class_0': 0.19934538006782532, 'iou_class_1': 0.1998450756072998, 'iou_class_2': 0.20040856301784515, 'iou_mean': 0.19986633956432343, 'precision_class_0': 0.33249780535697937, 'precision_class_1': 0.3331952393054962, 'precision_class_2': 0.3337491750717163, 'precision_mean': 0.33314740657806396, 'recall_class_0': 0.3323495388031006, 'recall_class_1': 0.33304107189178467, 'recall_class_2': 0.33405211567878723, 'recall_mean': 0.3331475555896759, 'dice': 0.33314743638038635, 'iou': 0.19986633956432343, 'precision': 0.33314740657806396, 'recall': 0.3331475555896759}


In [68]:
# Slice 2D
slice_pred = torch.rand(1, 3, 256, 256)  # logits
slice_true = torch.randint(0, 3, (1, 256, 256))  # índices de clase

# Volumen 3D 
volume_pred = torch.rand(1, 3, 64, 256, 256)  # logits
volume_true = torch.randint(0, 3, (1, 64, 256, 256))  # índices de clase

# Calcular métricas
slice_metrics = SegmentationMetrics.all_metrics(slice_pred, slice_true)
volume_metrics = SegmentationMetrics.all_metrics(volume_pred, volume_true)

print(volume_metrics)

{'dice_class_0': 1.932631254196167, 'dice_class_1': 1.9320595264434814, 'dice_class_2': 1.9491784572601318, 'dice_mean': 1.93795645236969, 'iou_class_0': 28.687334060668945, 'iou_class_1': 28.437543869018555, 'iou_class_2': 38.353416442871094, 'iou_mean': 31.82609748840332, 'precision_class_0': 0.9964141249656677, 'precision_class_1': 0.99611496925354, 'precision_class_2': 1.0052077770233154, 'precision_mean': 0.9992456436157227, 'recall_class_0': 31.98992919921875, 'recall_class_1': 31.985050201416016, 'recall_class_2': 31.995807647705078, 'recall_mean': 31.990262985229492, 'dice': 1.93795645236969, 'iou': 31.82609748840332, 'precision': 0.9992456436157227, 'recall': 31.990262985229492}


In [77]:
import torch
import numpy as np
from monai.metrics import DiceMetric, HausdorffDistanceMetric
from monai.data import decollate_batch

def validate_metrics_against_monai(y_pred, y_true):
    """
    Compare metrics between custom implementation and MONAI
    
    Parameters
    ----------
    y_pred : torch.Tensor
        Segmentation predictions
    y_true : torch.Tensor
        Ground truth masks
    
    Returns
    -------
    dict
        Metric comparison
    """
    # Ensure correct tensor format
    if y_pred.dim() == 4:  # [B, C, H, W]
        y_pred_argmax = torch.argmax(y_pred, dim=1)
        # Add channel dimension for MONAI metrics
        y_pred_argmax = y_pred_argmax.unsqueeze(1)
        y_true = y_true.unsqueeze(1)
    elif y_pred.dim() == 5:  # [B, C, D, H, W]
        y_pred_argmax = torch.argmax(y_pred, dim=1)
        # Add channel dimension for MONAI metrics
        y_pred_argmax = y_pred_argmax.unsqueeze(1)
        y_true = y_true.unsqueeze(1)
    else:
        y_pred_argmax = y_pred

    # Custom metrics
    custom_metrics = SegmentationMetrics.all_metrics(y_pred, y_true)

    # MONAI Metrics
    monai_dice_metric = DiceMetric(include_background=True)
    monai_dice_metric(y_pred_argmax, y_true)
    monai_dice = monai_dice_metric.aggregate().cpu().numpy()

    # Hausdorff Distance
    try:
        hausdorff_metric = HausdorffDistanceMetric(include_background=False)
        hausdorff_metric(y_pred_argmax, y_true)
        hausdorff_distance = hausdorff_metric.aggregate().cpu().numpy()
    except Exception as e:
        print(f"Hausdorff distance computation failed: {e}")
        hausdorff_distance = None

    # Comparison
    comparison = {
        'custom_metrics': custom_metrics,
        'monai_dice': monai_dice,
        'hausdorff_distance': hausdorff_distance
    }
    return comparison

# Ejemplo de uso
def test_metric_comparison():
    # Generar datos de ejemplo
    batch_size = 1
    n_classes = 3
    
    # 2D ejemplo
    slice_pred = torch.rand(batch_size, n_classes, 256, 256)
    slice_true = torch.randint(0, n_classes, (batch_size, 256, 256))

    # 3D ejemplo
    volume_pred = torch.rand(batch_size, n_classes, 64, 256, 256)
    volume_true = torch.randint(0, n_classes, (batch_size, 64, 256, 256))

    print("Métricas 2D:")
    slice_comparison = validate_metrics_against_monai(slice_pred, slice_true)
    print(slice_comparison)

    print("\nMétricas 3D:")
    volume_comparison = validate_metrics_against_monai(volume_pred, volume_true)
    print(volume_comparison)

In [80]:
# Descomentar para ejecutar pruebas
test_metric_comparison()

Métricas 2D:
{'custom_metrics': {'dice_class_0': 0.3325389623641968, 'dice_class_1': 0.3346504271030426, 'dice_class_2': 0.3304084837436676, 'dice_mean': 0.3325326144695282, 'iou_class_0': 0.19942831993103027, 'iou_class_1': 0.2009490579366684, 'iou_class_2': 0.1978977918624878, 'iou_mean': 0.19942505657672882, 'precision_class_0': 0.3337012529373169, 'precision_class_1': 0.33304542303085327, 'precision_class_2': 0.33085569739341736, 'precision_mean': 0.3325341045856476, 'recall_class_0': 0.33138471841812134, 'recall_class_1': 0.3362709879875183, 'recall_class_2': 0.32996249198913574, 'recall_mean': 0.3325394093990326, 'dice': 0.3325326144695282, 'iou': 0.19942505657672882, 'precision': 0.3325341045856476, 'recall': 0.3325394093990326}, 'monai_dice': array([0.33465043], dtype=float32), 'hausdorff_distance': array([3.1622777], dtype=float32)}

Métricas 3D:
{'custom_metrics': {'dice_class_0': 0.3337732255458832, 'dice_class_1': 0.33322998881340027, 'dice_class_2': 0.3336259722709656, 'di

In [79]:
import torch
import numpy as np
from monai.metrics import DiceMetric, HausdorffDistanceMetric

def validate_metrics(outputs, masks):
    """
    Validar y comparar métricas de segmentación
    
    Parameters
    ----------
    outputs : torch.Tensor
        Predicciones de segmentación [B, C, H, W]
    masks : torch.Tensor
        Máscaras ground truth [B, H, W]
    
    Returns
    -------
    dict
        Comparación de métricas
    """
    # Convertir outputs a predicciones de clase
    # Método 1: Argmax para obtener clase predicha
    y_pred_argmax = torch.argmax(outputs, dim=1)
    
    # Método 2: Softmax para obtener probabilidades de clase
    y_pred_softmax = torch.softmax(outputs, dim=1)
    
    # Métricas personalizadas 
    custom_metrics = SegmentationMetrics.all_metrics(outputs, masks)
    
    # Configurar métricas MONAI
    # Añadir dimensión de canal para MONAI
    y_pred_argmax_channel = y_pred_argmax.unsqueeze(1)
    masks_channel = masks.unsqueeze(1)
    
    # Métrica Dice de MONAI
    dice_metric = DiceMetric(include_background=False)
    dice_metric(y_pred_argmax_channel, masks_channel)
    monai_dice = dice_metric.aggregate().cpu().numpy()
    
    # Métrica Hausdorff de MONAI (opcional)
    try:
        hausdorff_metric = HausdorffDistanceMetric(include_background=False)
        hausdorff_metric(y_pred_argmax_channel, masks_channel)
        hausdorff_distance = hausdorff_metric.aggregate().cpu().numpy()
    except Exception as e:
        print(f"Error en Hausdorff: {e}")
        hausdorff_distance = None
    
    # Comparación de métricas
    comparison = {
        'custom_metrics': custom_metrics,
        'monai_dice': monai_dice,
        'hausdorff_distance': hausdorff_distance,
        # Información adicional de depuración
        'custom_dice_mean': custom_metrics.get('dice_mean'),
        'monai_dice_value': monai_dice[0] if monai_dice is not None else None
    }
    
    return comparison

# Ejemplo de uso
def test_metrics_comparison(outputs, masks):
    """
    Función de prueba para comparación de métricas
    """
    # Generar datos de ejemplo si no se proporcionan
    if outputs is None:
        batch_size, n_classes, height, width = 4, 5, 256, 256
        outputs = torch.rand(batch_size, n_classes, height, width)
        masks = torch.randint(0, n_classes, (batch_size, height, width))
    
    # Ejecutar comparación de métricas
    result = validate_metrics(outputs, masks)
    
    # Imprimir resultados
    print("Comparación de Métricas:")
    for key, value in result.items():
        print(f"{key}: {value}")
    
    return result

In [90]:
# Crear valores aleatorios simulando segmentaciones (batch_size=4, clases=5, 512x512)
y_pred = torch.randint(0, 5, (4, 512, 512))  # Predicción como índices de clase
y_true = torch.randint(0, 5, (4, 512, 512))  # Ground truth como índices de clase

# Convertir a one-hot para ambas implementaciones
n_classes = 5
y_pred_one_hot = F.one_hot(y_pred, num_classes=n_classes).permute(0, 3, 1, 2).float()
y_true_one_hot = F.one_hot(y_true, num_classes=n_classes).permute(0, 3, 1, 2).float()

# ======================
# Cálculo con MONAI
# ======================
dice_metric = DiceMetric(include_background=True, reduction="none")  # "none" para obtener todas las clases
monai_dice = dice_metric(y_pred_one_hot, y_true_one_hot)

# ======================
# Cálculo con tu implementación
# ======================
_, my_dice = sm2d.dice_coefficient(y_pred, y_true)  # Devuelve valores por clase en un diccionario

# ======================
# Comparación
# ======================
print("==> Comparación de Dice Score por clase:")
for i in range(n_classes):
    print(f"Clase {i}: MONAI = {monai_dice[:, i].mean().item():.8f}, Mi implementación = {my_dice[f'dice_class_{i}']:.8f}")

print(f"\n==> Dice medio:")
print(f"MONAI = {monai_dice.mean().item():.8f}, Mi implementación = {my_dice['dice_mean']:.8f}")

==> Comparación de Dice Score por clase:
Clase 0: MONAI = 0.19840868, Mi implementación = 0.19841214
Clase 1: MONAI = 0.20000958, Mi implementación = 0.20000857
Clase 2: MONAI = 0.19887729, Mi implementación = 0.19887893
Clase 3: MONAI = 0.19966599, Mi implementación = 0.19966757
Clase 4: MONAI = 0.20003915, Mi implementación = 0.20004581

==> Dice medio:
MONAI = 0.19940014, Mi implementación = 0.19940260


In [91]:
metrics_sm = sm2d.all_metrics(y_pred, y_true)
metrics_test = SegmentationMetrics.all_metrics(y_pred, y_true)
compare_metrics(metrics_sm, metrics_test)

Both metrics are identical.
Verification completed


# __Test 2__

In [113]:
def dice_coefficient(y_pred, y_true, smooth=1e-8):
    """
    Calcular coeficiente Dice para tensores 2D o 3D
    
    Parameters
    ----------
    y_pred : torch.Tensor
        Predicciones de segmentación [B, H, W] o [B, D, H, W]
    y_true : torch.Tensor
        Ground truth [B, H, W] o [B, D, H, W]
    smooth : float, optional
        Término de suavizado para evitar división por cero
    
    Returns
    -------
    dict
        Métricas Dice por clase
    """
    # Obtener número de clases
    n_classes = torch.max(torch.cat([y_pred, y_true])).item() + 1
    
    # Inicializar diccionario de métricas
    metrics = {}
    dice_scores = []
    
    # Calcular Dice por clase
    for c in range(n_classes):
        # Crear máscaras de clase
        pred_class = (y_pred == c).float()
        true_class = (y_true == c).float()
        
        # Calcular intersección y unión
        # Usar sum reduciendo todas las dimensiones excepto el batch
        intersection = torch.sum(pred_class * true_class, dim=(-3, -2, -1))
        union = torch.sum(pred_class, dim=(-3, -2, -1)) + torch.sum(true_class, dim=(-3, -2, -1))
        
        # Calcular Dice por muestra y promediar
        dice = (2 * intersection + smooth) / (union + smooth)
        dice_per_sample = dice.mean()
        
        # Almacenar métricas
        metrics[f'dice_class_{c}'] = dice_per_sample.item()
        dice_scores.append(dice_per_sample)
    
    # Calcular métricas agregadas
    mean_dice = torch.mean(torch.stack(dice_scores))
    metrics['dice_mean'] = mean_dice.item()
    
    return mean_dice, metrics

In [102]:
# 2D
y_pred_2d = torch.randint(0, 5, (4, 512, 512))
y_true_2d = torch.randint(0, 5, (4, 512, 512))

# 3D
y_pred_3d = torch.randint(0, 5, (4, 64, 512, 512))
y_true_3d = torch.randint(0, 5, (4, 64, 512, 512))

# Ambos funcionarán
_, metrics_2d = dice_coefficient(y_pred_2d, y_true_2d)
_, metrics_3d = dice_coefficient(y_pred_3d, y_true_3d)

In [114]:
import torch
import torch.nn.functional as F
from monai.metrics import DiceMetric

# Crear valores aleatorios simulando segmentaciones 3D
# Formato: (batch_size, depth, height, width)
y_pred = torch.randint(0, 5, (4, 64, 512, 512))  # Predicción como índices de clase
y_true = torch.randint(0, 5, (4, 64, 512, 512))  # Ground truth como índices de clase

# Número de clases
n_classes = 5

# Convertir a one-hot para ambas implementaciones
y_pred_one_hot = F.one_hot(y_pred, num_classes=n_classes).permute(0, 4, 1, 2, 3).float()
y_true_one_hot = F.one_hot(y_true, num_classes=n_classes).permute(0, 4, 1, 2, 3).float()

# ======================
# Cálculo con MONAI
# ======================
dice_metric = DiceMetric(include_background=True, reduction="none")  # "none" para obtener todas las clases
monai_dice = dice_metric(y_pred_one_hot, y_true_one_hot)

# ======================
# Cálculo con tu implementación
# ======================
# Asume que tienes un método similar sm3d.dice_coefficient() para 3D
_, my_dice = dice_coefficient(y_pred, y_true)  # Devuelve valores por clase en un diccionario

# ======================
# Comparación
# ======================
print("==> Comparación de Dice Score por clase:")
for i in range(n_classes):
    print(f"Clase {i}: MONAI = {monai_dice[:, i].mean().item():.8f}, Mi implementación = {my_dice[f'dice_class_{i}']:.8f}")

print(f"\n==> Dice medio:")
print(f"MONAI = {monai_dice.mean().item():.8f}, Mi implementación = {my_dice['dice_mean']:.8f}")

==> Comparación de Dice Score por clase:
Clase 0: MONAI = 0.19997947, Mi implementación = 0.19997947
Clase 1: MONAI = 0.19998191, Mi implementación = 0.19998191
Clase 2: MONAI = 0.20013496, Mi implementación = 0.20013496
Clase 3: MONAI = 0.20008358, Mi implementación = 0.20008358
Clase 4: MONAI = 0.20000929, Mi implementación = 0.20000929

==> Dice medio:
MONAI = 0.20003787, Mi implementación = 0.20003787


In [115]:
# Crear valores aleatorios simulando segmentaciones (batch_size=4, clases=5, 512x512)
y_pred = torch.randint(0, 5, (4, 512, 512))  # Predicción como índices de clase
y_true = torch.randint(0, 5, (4, 512, 512))  # Ground truth como índices de clase

# Convertir a one-hot para ambas implementaciones
n_classes = 5
y_pred_one_hot = F.one_hot(y_pred, num_classes=n_classes).permute(0, 3, 1, 2).float()
y_true_one_hot = F.one_hot(y_true, num_classes=n_classes).permute(0, 3, 1, 2).float()

# ======================
# Cálculo con MONAI
# ======================
dice_metric = DiceMetric(include_background=True, reduction="none")  # "none" para obtener todas las clases
monai_dice = dice_metric(y_pred_one_hot, y_true_one_hot)

# ======================
# Cálculo con tu implementación
# ======================
_, my_dice = dice_coefficient(y_pred, y_true)  # Devuelve valores por clase en un diccionario

# ======================
# Comparación
# ======================
print("==> Comparación de Dice Score por clase:")
for i in range(n_classes):
    print(f"Clase {i}: MONAI = {monai_dice[:, i].mean().item():.8f}, Mi implementación = {my_dice[f'dice_class_{i}']:.8f}")

print(f"\n==> Dice medio:")
print(f"MONAI = {monai_dice.mean().item():.8f}, Mi implementación = {my_dice['dice_mean']:.8f}")

==> Comparación de Dice Score por clase:
Clase 0: MONAI = 0.20061195, Mi implementación = 0.20061223
Clase 1: MONAI = 0.19966543, Mi implementación = 0.19966929
Clase 2: MONAI = 0.20014015, Mi implementación = 0.20014213
Clase 3: MONAI = 0.20003018, Mi implementación = 0.20002958
Clase 4: MONAI = 0.20014612, Mi implementación = 0.20014718

==> Dice medio:
MONAI = 0.20011875, Mi implementación = 0.20012009


# __Test 3__

In [170]:
from src.metrics.segmentation3d import SegmentationMetrics3D as sm3dA

In [172]:
# Example shapes
outputs = torch.randn(1, 5, 128, 128, 64)  # logits
masks = torch.randint(0, 5, (1, 128, 128, 64))  # ground truth class indices

metrics = sm3dA.all_metrics(outputs, masks)

In [173]:
print(metrics)

{'dice_class_0': 0.19993983209133148, 'dice_class_1': 0.20089128613471985, 'dice_class_2': 0.19933956861495972, 'dice_class_3': 0.2012404203414917, 'dice_class_4': 0.19956935942173004, 'dice_mean': 0.2001960724592209, 'iou_class_0': 0.11107397824525833, 'iou_class_1': 0.11166156083345413, 'iou_class_2': 0.11070358753204346, 'iou_class_3': 0.11187732964754105, 'iou_class_4': 0.11084534972906113, 'iou_mean': 0.11123236268758774, 'precision_class_0': 0.20000235736370087, 'precision_class_1': 0.20098738372325897, 'precision_class_2': 0.19934915006160736, 'precision_class_3': 0.2013186365365982, 'precision_class_4': 0.19932405650615692, 'precision_mean': 0.2001963108778, 'recall_class_0': 0.19987736642360687, 'recall_class_1': 0.2007952779531479, 'recall_class_2': 0.19932997226715088, 'recall_class_3': 0.20116226375102997, 'recall_class_4': 0.1998152732849121, 'recall_mean': 0.2001960277557373, 'dice': 0.2001960724592209, 'iou': 0.11123236268758774, 'precision': 0.2001963108778, 'recall': 0

In [189]:
import torch
import numpy as np
from monai.metrics import DiceMetric

def compare_dice_metrics():
    # Establecer una semilla para reproducibilidad
    # torch.manual_seed(42)
    # np.random.seed(42)

    # Generar datos de ejemplo
    # outputs: logits de predicción [B, C, H, W, D]
    # masks: máscaras de ground truth [B, H, W, D]
    batch_size = 1
    num_classes = 5
    height, width, depth = 128, 128, 64

    # Generar predicciones y máscaras aleatorias
    outputs = torch.randn(batch_size, num_classes, height, width, depth)
    masks_indices = torch.randint(0, num_classes, (batch_size, height, width, depth))

    # Calcular Dice con nuestra implementación
    mean_dice_custom, class_dice_custom = sm3dA.dice_coefficient(outputs, masks_indices)

    # Calcular Dice con MONAI
    # Necesitamos convertir los datos al formato correcto para MONAI
    # MONAI usa softmax en los logits
    outputs_softmax = torch.nn.functional.softmax(outputs, dim=1)
    
    # Convertir máscaras a one-hot para MONAI
    masks_one_hot = torch.nn.functional.one_hot(masks_indices, num_classes=num_classes).permute(0, 4, 1, 2, 3).float()

    # Usar DiceMetric de MONAI
    dice_metric = DiceMetric(include_background=True, reduction='mean')
    dice_metric(outputs_softmax, masks_one_hot)
    monai_dice = dice_metric.aggregate().item()

    # Imprimir resultados
    print("Nuestra implementación:")
    print("Mean Dice:", mean_dice_custom.item())
    print("Clase por clase:", class_dice_custom)
    
    print("\nMONAI:")
    print("Mean Dice:", monai_dice)

    # Comparar resultados
    print("\nDiferencia absoluta:", abs(mean_dice_custom.item() - monai_dice))
    
    # Comparación de Dice por clase (si es posible)
    print("\nComprobación de consistencia:")
    print("¿Los resultados son muy similares?", 
          np.allclose(mean_dice_custom.item(), monai_dice, atol=1e-5))


In [190]:
compare_dice_metrics()

Nuestra implementación:
Mean Dice: 0.20002751052379608
Clase por clase: {'dice_class_0': 0.20039300620555878, 'dice_class_1': 0.20016027987003326, 'dice_class_2': 0.20015661418437958, 'dice_class_3': 0.19990874826908112, 'dice_class_4': 0.19951891899108887, 'dice_mean': 0.20002751052379608}

MONAI:
Mean Dice: 0.33333325386047363

Diferencia absoluta: 0.13330574333667755

Comprobación de consistencia:
¿Los resultados son muy similares? False


In [193]:
def compare_dice_metrics():
    # Parámetros
    batch_size = 1
    num_classes = 5
    height, width, depth = 128, 128, 64

    # Generar predicciones como logits más realistas
    # Usando una distribución que simule probabilidades softmax
    outputs = torch.randn(batch_size, num_classes, height, width, depth)
    
    # Crear máscaras one-hot más controladas
    # En lugar de aleatorio, crear una distribución más realista
    masks_indices = torch.zeros(batch_size, height, width, depth, dtype=torch.long)
    
    # Simular una máscara más realista
    # Por ejemplo, asignar una clase dominante en diferentes regiones
    masks_indices[:, :height//2, :width//2, :depth//2] = 1
    masks_indices[:, height//2:, width//2:, depth//2:] = 2
    
    # Convertir máscaras a formato adecuado para MONAI
    masks_one_hot = torch.nn.functional.one_hot(
        masks_indices, 
        num_classes=num_classes
    ).permute(0, 4, 1, 2, 3).float()

    # Calcular con nuestra implementación
    outputs_softmax = torch.nn.functional.softmax(outputs, dim=1)
    mean_dice_custom, class_dice_custom = sm3dA.dice_coefficient(outputs, masks_indices)

    # Calcular con MONAI
    dice_metric = DiceMetric(include_background=True, reduction='mean')
    dice_metric(outputs_softmax, masks_one_hot)
    monai_dice = dice_metric.aggregate().item()

    # Imprimir resultados
    print("Nuestra implementación:")
    print("Mean Dice:", mean_dice_custom.item())
    print("Clase por clase:", class_dice_custom)
    
    print("\nMONAI:")
    print("Mean Dice:", monai_dice)

    # Comparar resultados
    print("\nDiferencia absoluta:", abs(mean_dice_custom.item() - monai_dice))
    print("¿Resultados muy similares?", 
          np.isclose(mean_dice_custom.item(), monai_dice, atol=1e-5))

In [194]:
for i in range(3):
    print(f"\n--- Iteración {i+1} ---")
    compare_dice_metrics()


--- Iteración 1 ---
Nuestra implementación:
Mean Dice: 0.12478236854076385
Clase por clase: {'dice_class_0': 0.31600162386894226, 'dice_class_1': 0.15377706289291382, 'dice_class_2': 0.15413318574428558, 'dice_class_3': 4.763049329947755e-12, 'dice_class_4': 4.770744996968057e-12, 'dice_mean': 0.12478236854076385}

MONAI:
Mean Dice: 0.43386244773864746

Diferencia absoluta: 0.3090800791978836
¿Resultados muy similares? False

--- Iteración 2 ---
Nuestra implementación:
Mean Dice: 0.1246698647737503
Clase por clase: {'dice_class_0': 0.31591299176216125, 'dice_class_1': 0.1541236937046051, 'dice_class_2': 0.15331260859966278, 'dice_class_3': 4.769694621903353e-12, 'dice_class_4': 4.762880194408847e-12, 'dice_mean': 0.1246698647737503}

MONAI:
Mean Dice: 0.43386244773864746

Diferencia absoluta: 0.30919258296489716
¿Resultados muy similares? False

--- Iteración 3 ---
Nuestra implementación:
Mean Dice: 0.12445126473903656
Clase por clase: {'dice_class_0': 0.31577351689338684, 'dice_class

In [195]:
import torch
import numpy as np
from monai.metrics import DiceMetric

class SegmentationMetrics3D:
    @staticmethod
    def dice_coefficient(y_pred, y_true, include_background=True, smooth=1e-6):
        """
        Implementación de Dice coefficient similar a MONAI
        """
        # Asegurar softmax
        y_pred_softmax = torch.nn.functional.softmax(y_pred, dim=1)
        
        # Convertir a one-hot si es necesario
        if y_true.dim() == 4:
            y_true_one_hot = torch.nn.functional.one_hot(
                y_true, 
                num_classes=y_pred.size(1)
            ).permute(0, 4, 1, 2, 3).float()
        else:
            y_true_one_hot = y_true
        
        # Calcular Dice para cada clase
        dice_scores = []
        class_dice = {}
        
        start_class = 0 if include_background else 1
        
        for i in range(start_class, y_pred_softmax.size(1)):
            pred_class = y_pred_softmax[:, i, :, :, :]
            true_class = y_true_one_hot[:, i, :, :, :]
            
            # Cálculo de Dice más robusto
            intersection = torch.sum(pred_class * true_class)
            pred_sum = torch.sum(pred_class)
            true_sum = torch.sum(true_class)
            
            dice = (2.0 * intersection + smooth) / (pred_sum + true_sum + smooth)
            
            dice_scores.append(dice)
            class_dice[f"dice_class_{i}"] = dice.item()
        
        # Promedio de Dice excluyendo clases con cero
        mean_dice = torch.mean(torch.stack(dice_scores))
        class_dice["dice_mean"] = mean_dice.item()
        
        return mean_dice, class_dice

def compare_dice_metrics():
    # Parámetros
    batch_size = 1
    num_classes = 5
    height, width, depth = 128, 128, 64

    # Generar predicciones como logits más realistas
    outputs = torch.randn(batch_size, num_classes, height, width, depth)
    
    # Crear máscaras one-hot más controladas
    masks_indices = torch.zeros(batch_size, height, width, depth, dtype=torch.long)
    
    # Simular una máscara más realista
    masks_indices[:, :height//2, :width//2, :depth//2] = 1
    masks_indices[:, height//2:, width//2:, depth//2:] = 2
    
    # Convertir máscaras a formato adecuado para MONAI
    masks_one_hot = torch.nn.functional.one_hot(
        masks_indices, 
        num_classes=num_classes
    ).permute(0, 4, 1, 2, 3).float()

    # Calcular con nuestra implementación
    outputs_softmax = torch.nn.functional.softmax(outputs, dim=1)
    mean_dice_custom, class_dice_custom = SegmentationMetrics3D.dice_coefficient(
        outputs, masks_indices, include_background=True
    )

    # Calcular con MONAI
    dice_metric = DiceMetric(include_background=True, reduction='mean')
    dice_metric(outputs_softmax, masks_one_hot)
    monai_dice = dice_metric.aggregate().item()

    # Imprimir resultados
    print("Nuestra implementación:")
    print("Mean Dice:", mean_dice_custom.item())
    print("Clase por clase:", class_dice_custom)
    
    print("\nMONAI:")
    print("Mean Dice:", monai_dice)

    # Comparar resultados
    print("\nDiferencia absoluta:", abs(mean_dice_custom.item() - monai_dice))
    print("¿Resultados muy similares?", 
          np.isclose(mean_dice_custom.item(), monai_dice, atol=1e-4))

# Ejecutar múltiples veces
for i in range(3):
    print(f"\n--- Iteración {i+1} ---")
    compare_dice_metrics()


--- Iteración 1 ---
Nuestra implementación:
Mean Dice: 0.12471222877502441
Clase por clase: {'dice_class_0': 0.31604287028312683, 'dice_class_1': 0.15355075895786285, 'dice_class_2': 0.15396748483181, 'dice_class_3': 4.769620462474755e-12, 'dice_class_4': 4.764668694312579e-12, 'dice_mean': 0.12471222877502441}

MONAI:
Mean Dice: 0.43386244773864746

Diferencia absoluta: 0.30915021896362305
¿Resultados muy similares? False

--- Iteración 2 ---
Nuestra implementación:
Mean Dice: 0.12457618862390518
Clase por clase: {'dice_class_0': 0.3158639371395111, 'dice_class_1': 0.15336500108242035, 'dice_class_2': 0.15365198254585266, 'dice_class_3': 4.766590334243093e-12, 'dice_class_4': 4.769336835186433e-12, 'dice_mean': 0.12457618862390518}

MONAI:
Mean Dice: 0.43386244773864746

Diferencia absoluta: 0.3092862591147423
¿Resultados muy similares? False

--- Iteración 3 ---
Nuestra implementación:
Mean Dice: 0.12456078827381134
Clase por clase: {'dice_class_0': 0.3154163658618927, 'dice_class_1

In [196]:
import torch
import numpy as np
from monai.metrics import DiceMetric

class SegmentationMetrics3D:
    @staticmethod
    def dice_coefficient(y_pred, y_true, include_background=True, smooth=1e-6):
        """
        Improved Dice coefficient implementation compatible with MONAI
        
        Args:
            y_pred (torch.Tensor): Predicted logits/probabilities
            y_true (torch.Tensor): Ground truth labels
            include_background (bool): Whether to include background class
            smooth (float): Smoothing factor to prevent division by zero
        
        Returns:
            tuple: Mean Dice coefficient and per-class Dice scores
        """
        # Ensure prediction is in probability space
        y_pred_softmax = torch.nn.functional.softmax(y_pred, dim=1)
        
        # Convert ground truth to one-hot encoding if necessary
        if y_true.dim() == 4:  # If input is class indices
            y_true_one_hot = torch.nn.functional.one_hot(
                y_true, 
                num_classes=y_pred.size(1)
            ).permute(0, 4, 1, 2, 3).float()
        else:  # If already one-hot encoded
            y_true_one_hot = y_true
        
        # Compute Dice for each class
        dice_scores = []
        class_dice = {}
        
        # Determine starting class based on include_background
        start_class = 0 if include_background else 1
        
        for i in range(start_class, y_pred_softmax.size(1)):
            # Select class-specific predictions and ground truth
            pred_class = y_pred_softmax[:, i, :, :, :]
            true_class = y_true_one_hot[:, i, :, :, :]
            
            # Compute intersection and totals
            intersection = torch.sum(pred_class * true_class)
            pred_sum = torch.sum(pred_class)
            true_sum = torch.sum(true_class)
            
            # Compute Dice coefficient with smoothing
            dice = (2.0 * intersection + smooth) / (pred_sum + true_sum + smooth)
            
            dice_scores.append(dice)
            class_dice[f"dice_class_{i}"] = dice.item()
        
        # Compute mean Dice score
        mean_dice = torch.mean(torch.stack(dice_scores))
        class_dice["dice_mean"] = mean_dice.item()
        
        return mean_dice, class_dice

def compare_dice_metrics():
    # Parameters
    batch_size = 1
    num_classes = 5
    height, width, depth = 128, 128, 64
    
    # Generate more controlled predictions and ground truth
    outputs = torch.randn(batch_size, num_classes, height, width, depth)
    
    # Create a more realistic mask
    masks_indices = torch.zeros(batch_size, height, width, depth, dtype=torch.long)
    
    # Simulate different regions for different classes
    masks_indices[:, :height//2, :width//2, :depth//2] = 1
    masks_indices[:, height//2:, width//2:, depth//2:] = 2
    masks_indices[:, :height//4, width//4:3*width//4, depth//4:3*depth//4] = 3
    
    # Compute metrics with both methods
    # Custom implementation
    mean_dice_custom, class_dice_custom = SegmentationMetrics3D.dice_coefficient(
        outputs, masks_indices, include_background=True
    )
    
    # MONAI implementation
    # Prepare one-hot encoding for MONAI
    masks_one_hot = torch.nn.functional.one_hot(
        masks_indices, 
        num_classes=num_classes
    ).permute(0, 4, 1, 2, 3).float()
    
    # Outputs softmax for MONAI
    outputs_softmax = torch.nn.functional.softmax(outputs, dim=1)
    
    # MONAI Dice metric with per-class computation
    dice_metric = DiceMetric(include_background=True, reduction='mean')
    dice_metric(outputs_softmax, masks_one_hot)
    
    # Aggregate and get per-class metrics
    monai_dice = dice_metric.aggregate().item()
    
    # Compute per-class MONAI Dice manually for verification
    monai_per_class_dice = {}
    for i in range(num_classes):
        pred_class = outputs_softmax[:, i, :, :, :]
        true_class = masks_one_hot[:, i, :, :, :]
        
        intersection = torch.sum(pred_class * true_class)
        pred_sum = torch.sum(pred_class)
        true_sum = torch.sum(true_class)
        
        dice = (2.0 * intersection + 1e-6) / (pred_sum + true_sum + 1e-6)
        monai_per_class_dice[f"dice_class_{i}"] = dice.item()
    
    monai_per_class_dice["dice_mean"] = monai_dice
    
    # Print results
    print("Nuestra implementación:")
    print("Mean Dice:", mean_dice_custom.item())
    print("Clase por clase:", class_dice_custom)
    
    print("\nMONAI:")
    print("Mean Dice:", monai_dice)
    print("Clase por clase:", monai_per_class_dice)
    
    # Compare results
    print("\nDiferencia absoluta:", abs(mean_dice_custom.item() - monai_dice))
    print("¿Resultados muy similares?", 
          np.isclose(mean_dice_custom.item(), monai_dice, atol=1e-4))

# Run multiple iterations
for i in range(3):
    print(f"\n--- Iteración {i+1} ---")
    compare_dice_metrics()


--- Iteración 1 ---
Nuestra implementación:
Mean Dice: 0.14040404558181763
Clase por clase: {'dice_class_0': 0.31157687306404114, 'dice_class_1': 0.14111420512199402, 'dice_class_2': 0.15371327102184296, 'dice_class_3': 0.09561585634946823, 'dice_class_4': 4.772596814278662e-12, 'dice_mean': 0.14040404558181763}

MONAI:
Mean Dice: 0.340685099363327
Clase por clase: {'dice_class_0': 0.31157687306404114, 'dice_class_1': 0.14111420512199402, 'dice_class_2': 0.15371327102184296, 'dice_class_3': 0.09561585634946823, 'dice_class_4': 4.772596814278662e-12, 'dice_mean': 0.340685099363327}

Diferencia absoluta: 0.2002810537815094
¿Resultados muy similares? False

--- Iteración 2 ---
Nuestra implementación:
Mean Dice: 0.14028918743133545
Clase por clase: {'dice_class_0': 0.31131622195243835, 'dice_class_1': 0.14149685204029083, 'dice_class_2': 0.15382051467895508, 'dice_class_3': 0.09481234848499298, 'dice_class_4': 4.766771612846332e-12, 'dice_mean': 0.14028918743133545}

MONAI:
Mean Dice: 0.3

In [202]:
import torch
import torch.nn.functional as F
from monai.metrics import DiceMetric

def compare_3d_dice_metrics():
    # Set random seed for reproducibility
    # torch.manual_seed(42)
    
    # Create random 3D segmentation data 
    # (batch_size=4, depth=64, height=512, width=512)
    batch_size, depth, height, width = 4, 64, 512, 512
    n_classes = 5
    
    y_pred = torch.randint(0, n_classes, (batch_size, depth, height, width))  # Prediction as class indices
    y_true = torch.randint(0, n_classes, (batch_size, depth, height, width))  # Ground truth as class indices
    
    # Convert to one-hot for MONAI
    y_pred_one_hot = F.one_hot(y_pred, num_classes=n_classes).permute(0, 4, 1, 2, 3).float()
    y_true_one_hot = F.one_hot(y_true, num_classes=n_classes).permute(0, 4, 1, 2, 3).float()
    
    # ======================
    # Calculate with MONAI
    # ======================
    dice_metric = DiceMetric(include_background=True, reduction="none")  # "none" to get per-class scores
    monai_dice = dice_metric(y_pred_one_hot, y_true_one_hot)
    
    # ======================
    # Calculate with custom implementation
    # ======================
    mean_dice, my_dice = SegmentationMetrics3D.dice_coefficient(y_pred, y_true)
    
    # ======================
    # Comparison
    # ======================
    print("==> Dice Score Comparison per Class:")
    for i in range(n_classes):
        print(f"Class {i}: MONAI = {monai_dice[:, i].mean().item():.6f}, Custom = {my_dice[f'dice_class_{i}']:.6f}")
    
    print(f"\n==> Mean Dice:")
    print(f"MONAI = {monai_dice.mean().item():.6f}, Custom = {my_dice['dice_mean']:.6f}")

    print()
    print(monai_dice)
    print()
    print(my_dice)

# Run comparison
compare_3d_dice_metrics()

==> Dice Score Comparison per Class:
Class 0: MONAI = 0.199830, Custom = 0.199830
Class 1: MONAI = 0.200171, Custom = 0.200171
Class 2: MONAI = 0.199945, Custom = 0.199945
Class 3: MONAI = 0.199953, Custom = 0.199953
Class 4: MONAI = 0.200077, Custom = 0.200077

==> Mean Dice:
MONAI = 0.199995, Custom = 0.199995

tensor([[0.1997, 0.2001, 0.2001, 0.1996, 0.1999],
        [0.2000, 0.2004, 0.1996, 0.2000, 0.2001],
        [0.2001, 0.2000, 0.2000, 0.1999, 0.2003],
        [0.1995, 0.2001, 0.2001, 0.2003, 0.2000]])

{'dice_class_0': 0.19983042776584625, 'dice_class_1': 0.2001706212759018, 'dice_class_2': 0.19994518160820007, 'dice_class_3': 0.19995322823524475, 'dice_class_4': 0.20007702708244324, 'dice_mean': 0.19999530911445618}


In [200]:
compare_3d_dice_metrics()

==> Dice Score Comparison per Class:
Class 0: MONAI = 0.2000, Custom = 0.2000
Class 1: MONAI = 0.2001, Custom = 0.2001
Class 2: MONAI = 0.2000, Custom = 0.2000
Class 3: MONAI = 0.2001, Custom = 0.2001
Class 4: MONAI = 0.2001, Custom = 0.2001

==> Mean Dice:
MONAI = 0.2001, Custom = 0.2001


# __Testtt__

In [208]:
import monai.metrics as mm
def dice_coefficient(y_pred, y_true, smooth=1e-6):
    # Convert to one-hot if inputs are class indices
    if y_pred.dim() == 3:
        # Convert predicted class indices to one-hot
        n_classes = torch.max(y_true).item() + 1
        y_pred_one_hot = torch.zeros(
            y_pred.size(0), n_classes, y_pred.size(1), y_pred.size(2), 
            device=y_pred.device
        )
        y_pred_one_hot.scatter_(1, y_pred.unsqueeze(1), 1)
        
        y_true_one_hot = torch.zeros(
            y_true.size(0), n_classes, y_true.size(1), y_true.size(2), 
            device=y_true.device
        )
        y_true_one_hot.scatter_(1, y_true.unsqueeze(1), 1)
    else:
        # If already in form [B, C, H, W] (logits or one-hot)
        if y_pred.dim() == 4 and y_true.dim() == 3:
            # y_pred is [B, C, H, W] logits and y_true is [B, H, W] indices
            n_classes = y_pred.size(1)
            y_pred_one_hot = torch.nn.functional.softmax(y_pred, dim=1)
            
            y_true_one_hot = torch.zeros(
                y_true.size(0), n_classes, y_true.size(1), y_true.size(2), 
                device=y_true.device
            )
            y_true_one_hot.scatter_(1, y_true.unsqueeze(1), 1)
        else:
            # Assume both are already in proper format
            y_pred_one_hot = y_pred
            y_true_one_hot = y_true
            n_classes = y_pred.size(1)

    dice_metric = mm.DiceMetric(include_background=True, reduction='none', num_classes=n_classes)
    dices = dice_metric(y_pred, y_true)
    print(dices)
    class_dice = {}
    for i in range(n_classes):
        # print(f"Class {i}: MONAI = {monai_dice[:, i].mean().item():.6f}, Custom = {my_dice[f'dice_class_{i}']:.6f}")
        class_dice[f"dice_class_{i}"] = dices[:, i].mean().item()

    return class_dice

In [211]:
batch_size, depth, height, width = 4, 64, 512, 512
n_classes = 5

y_pred = torch.randint(n_classes, batch_size, height, width)  # Prediction as class indices
y_true = torch.randint(n_classes, batch_size, height, width)  # Ground truth as class indices

print(y_pred.shape)
print(y_true.shape)
print(dice_coefficient(y_pred, y_true))
_, dc = sm2d.dice_coefficient(y_pred, y_true) 
print(dc)

TypeError: randint() received an invalid combination of arguments - got (int, int, int, int), but expected one of:
 * (int high, tuple of ints size, *, torch.Generator generator, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (int high, tuple of ints size, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (int low, int high, tuple of ints size, *, torch.Generator generator, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (int low, int high, tuple of ints size, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
